# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [3]:
import sqlite3
import pandas as pd

In [4]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [10]:
# Your code here
q = '''
SELECT
    CustomerNumber,
    ContactLastName,
    ContactFirstName
FROM Customers
WHERE CustomerNumber IN (
        SELECT CustomerNumber
        FROM Orders
        WHERE OrderDate = '2003-01-31');
        '''
pd.read_sql(q, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [27]:
# doing this with join
pd.read_sql('''
    SELECT p.productName, count(od.orderNumber) as OrderTotal
    FROM products p
    JOIN orderdetails od USING(productCode)
    GROUP BY productName
    ORDER BY OrderTotal DESC

''', conn)

,productName,OrderTotal
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
104,1932 Alfa Romeo 8C2300 Spider Sport,25
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24


In [30]:
#doing this with subqueries
pd.read_sql('''
 SELECT productName, (
     SELECT count(orderNumber)
     FROM orderdetails
     WHERE orderdetails.productCode = products.productCode
     ) AS OrdersTotal
 FROM products
 GROUP BY productName
 ORDER BY OrdersTotal DESC;
'''
,conn)


,productName,OrdersTotal
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24
108,1952 Citroen-15CV,24


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [56]:
# Your code here

# use DISTINCT to get the distinct number of customers who made an order
pd.read_sql('''
    SELECT productName,productCode,
         (SELECT count(DISTINCT customerNumber)
         FROM orderdetails od
         JOIN orders o USING(orderNumber)
         JOIN customers c USING(customerNumber)
         WHERE od.productCode = p.productCode
         
         ) AS num_of_customers
    FROM products p
    ORDER BY num_of_customers DESC

''', conn).head(20)

,productName,productCode,num_of_customers
0,1992 Ferrari 360 Spider red,S18_3232,40
1,1952 Alpine Renault 1300,S10_1949,27
2,1972 Alfa Romeo GTA,S10_4757,27
3,1934 Ford V8 Coupe,S18_2957,27
4,Boeing X-32A JSF,S72_1253,27
5,1969 Harley Davidson Ultimate Chopper,S10_1678,26
6,1996 Moto Guzzi 1100i,S10_2016,26
7,1980s Black Hawk Helicopter,S18_1662,26
8,1995 Honda Civic,S18_1984,26
9,1913 Ford Model T Speedster,S18_2949,26


In [50]:
## let me try this with a joi1n

pd.read_sql('''
    SELECT productName, productCode,count(DISTINCT customerNumber) AS num_of_customers
    FROM products
    JOIN orderdetails USING(productCode)
    JOIN orders USING(orderNumber)
    JOIN customers USING(customerNumber)
    GROUP BY productName
    ORDER BY num_of_customers DESC
''', conn).head(20)

,productName,productCode,num_of_customers
0,1992 Ferrari 360 Spider red,S18_3232,40
1,Boeing X-32A JSF,S72_1253,27
2,1972 Alfa Romeo GTA,S10_4757,27
3,1952 Alpine Renault 1300,S10_1949,27
4,1934 Ford V8 Coupe,S18_2957,27
5,HMS Bounty,S700_2047,26
6,Corsair F4U ( Bird Cage),S24_3949,26
7,American Airlines: MD-11S,S700_4002,26
8,1999 Yamaha Speed Boat,S18_3029,26
9,1996 Moto Guzzi 1100i,S10_2016,26


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [57]:
# am solving this using join 

pd.read_sql('''
    SELECT e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode
    FROM employees e
    JOIN offices o ON e.officeCode = o.officeCode
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders ord ON c.customerNumber = ord.customerNumber
    JOIN orderdetails od ON ord.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
    WHERE p.productCode IN (
    SELECT od2.productCode
    FROM orderdetails od2
    JOIN orders ord2 ON od2.orderNumber = ord2.orderNumber
    GROUP BY od2.productCode
    HAVING COUNT(DISTINCT ord2.customerNumber) < 20
)
GROUP BY e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode;

''',conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [58]:
# Your code here
pd.read_sql(
'''
    SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(c.customerNumber) AS num_of_customers
    FROM employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY e.employeeNumber, e.firstName, e.lastName
    HAVING AVG(c.creditLimit) > 15000;

''',conn)

,employeeNumber,firstName,lastName,num_of_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


In [ ]:
conn.close()

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!